In [5]:
# importing necessary libraries
import pandas as pd
import numpy as np

## 1. Loading your data

Use the pandas function read_excel() to load your data into python as a pandas DataFrame. A pandas DataFrame is similar to an excel file in that it has a tabular (table) structure, both have labels/column headers, and are great forms to store your data in for manipulation.

In [6]:
# Change the text in the function to the file path where you downloaded the xlsx file on your computer
# A file path is where a file lives on your computer, to find the file path on your machine, locate the file you want to use in your file manager (finder),
# right click the file (then hold down the option key if using Mac) and copy the file path. Easy!
data = pd.read_excel("/workspaces/audible_data_cleaning/audible_start.xlsx")

## 2. Getting to know your data

The pandas head() function is great to get a glimpse at your data when you load it in, as well as visually checking your data as you clean it and manipulate it! It returns the first 5 rows of your data (indices 0-4), and the inverse to this function is tail(). Try using tail to check the last 5 entries in your dataset.

In [7]:
data.head()

,ID,name,author,narrator,time,releasedate,language,stars,price
0,294217,Shattered Hearts (German edition),Writtenby:Anne-MarieJungwirth,"Narratedby:FundaVanroy,SebastianFischer",8 hrs and 47 mins,19-11-21,german,Not rated yet,401
1,253599,The Ultimate Colin Wilson,"Writtenby:ColinWilson,ColinStanley-editedby",Narratedby:LiamGerrard,15 hrs and 18 mins,14-05-19,USEnglish,Not rated yet,703
2,210584,Bloody Confused!,Writtenby:ChuckCulpepper,Narratedby:AlexHyde-White,9 hrs and 48 mins,2014-04-02 00:00:00,English,Not rated yet,668
3,236792,Operation Greylord,"Writtenby:TerrenceHake,WayneKlatt",Narratedby:CharlesConstant,10 hrs and 14 mins,2015-07-08 00:00:00,English,Not rated yet,891
4,261210,The Little Book of Main Street Money,Writtenby:JonathanClements,Narratedby:SeanPratt,4 hrs and 23 mins,15-07-09,English,5 out of 5 stars1 rating,703


Next, lets look at the datatypes stored in the columns of our dataframe. We will use the pandas dtypes attribute to return the datatype of each column in our dataframe. You have seen some functions from pandas, and now may be asking, "what is an attribute?"

An attribute is some characteristic of an object, an object being a pandas dataframe in our case. These are extremely useful when manipulating data in python! Other attributes include columns, size, index, shape, and more.

In [8]:
data.dtypes

ID              int64
name           object
author         object
narrator       object
time           object
releasedate    object
language       object
stars          object
price          object
dtype: object

Looking at the numbers of rows and columsn in our data

In [9]:
data.shape

(174978, 9)

After glancing at our data and checking the datatypes, it's clear we have some dirty data.

From first glance: columns like author and narrator have unnecessary text like "Writtenby:" and "Narratedby:". It is clear that we have inconsistent datetime forms in the releasedate column. And lastly, the stars column clearly needs some TLC.

From further datatype investigation: yikes. The ID column in an int which is bad because we do not want to perform mathematical calculations on ID since it's a primary key (unique identifier), so that datatype needs to change. The rest of the columns are all objects, meaning that they hold multiple different datatypes. Trying to perform an analysis on a column with multiple datatypes is extremely difficult.

Lets fix this!

## 3. Cleaning your data

The first thing I like to do when cleaning data is checking for duplicates. This code uses the pandas duplicated() function to find duplicates in our data. I initialize those a variable named "dupes" and then print the length using the built-in python function len() which returns the size of various python objects.

In [10]:
dupes = data[data.duplicated()]
print(len(dupes))

87489


Let's drop those duplicates using the drop_duplicates function from pandas.

In [11]:
data = data.drop_duplicates()

Next, I check for empty values in the dataframe and then drop them

In [12]:
nulls = data[data.isnull().any(axis=1)]
print(len(nulls))

491


In [13]:
data = data.dropna()

Let's convert the ID field to a str

In [14]:
data['ID'] = data['ID'].astype(str)

Now, let's fix the author and narrator columns by removing the unnecessary text.

In [15]:
# Remove Writtenby: by using pandas replace
data['author'] = data['author'].str.replace('Writtenby:', '')
# Same process for narrator column
data['narrator'] = data['narrator'].str.replace('Narratedby:', '')

Next step is fixing that inconsistent time column. Below I am writing a function that uses regex, short for regular expression, to find patterns in our text and then apply mathematical calcs based on the findings in the text. Revisit this code after more of your python classes and you'll realize that you can write functions like this too!

In [16]:
# I'm not going to explain this function in further depth because it's higher level but I promise you will look back at this in a year and 
# fully understand it! Keep coding!

def convert_time_to_minutes(df: pd.DataFrame):
    # Create a minutes column
    df['minutes'] = 0
    
    # Use regex to get hours and minutes
    hours = df['time'].str.extract(r'(\d+) hr?', expand=False).fillna(0).astype(int)
    minutes = df['time'].str.extract(r'(\d+) min', expand=False).fillna(0).astype(int)
    
    # Take care of the less than 1 minute thing
    less_than_minute_mask = df['time'].str.contains('less than 1 minute')
    
    # Simple math!
    df.loc[~less_than_minute_mask, 'minutes'] += hours * 60
    df.loc[~less_than_minute_mask, 'minutes'] += minutes
    df.loc[less_than_minute_mask, 'minutes'] += 1
    
    return df

Applying the function and dropping the dirty data column.

In [17]:
# Call the function with our dataframe
data = convert_time_to_minutes(data)

# Drop the time column
data = data.drop(columns=['time'])

Now let's fix the inconsistent dates. We want the dates in standard MM/DD/YY format that we use in the US. The code below takes the releasedate column and uses the pandas to_datetime() function to turn the column values into dates, and I then pass the dayfirst=True parameter to get all the values in the same order. The strftime() function then converts these values all to MM/DD/YY.

In [18]:
data['releasedate'] = pd.to_datetime(data['releasedate'], dayfirst=True, errors='coerce').dt.strftime('%m/%d/%y')

/tmp/ipykernel_5738/72039345.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['releasedate'] = pd.to_datetime(data['releasedate'], dayfirst=True, errors='coerce').dt.strftime('%m/%d/%y')


Below I am writing a function that again uses regex to break the stars column out to two separate columns, star_rating and ratings_count. Feel free to read through and try to understand the code, but don't feel discouraged if you don't grasp everything happening - you'll get there!

In [19]:
def extract_stars_and_ratings(df: pd.DataFrame, column_name: str):
    
    data[column_name] = data[column_name].astype(str)

    # Using regex to see where we want to break this column up
    stars_pattern = r'(\d) out of 5 stars'
    ratings_pattern = r'(\d+) rating|(\d+) ratings'
    
    # Extract stars and ratings to different columns
    df['star_rating'] = df[column_name].str.extract(stars_pattern, expand=False)
    df['star_rating'] = df['star_rating'].astype(float) if 'star_rating' in df.columns else np.nan
    
    # Extract ratings and stars
    ratings = df[column_name].str.extract(ratings_pattern, expand=False)
    df['ratings_count'] = ratings.apply(lambda row: int(row[0]) if pd.notnull(row[0]) else int(row[1]) if pd.notnull(row[1]) else 0, axis=1)
    
    # Handle "Not yet rated" and NaN values
    df.loc[df[column_name].str.contains('Not yet rated', na=False), 'star_rating'] = np.nan
    
    return df[['star_rating', 'ratings_count']]

Call the function and break out the stars and ratings.

In [20]:
data[['star_rating', 'ratings_count']] = extract_stars_and_ratings(data, 'stars')
data = data.drop(columns=['stars'])

Next we will fix the price column. Some values are ints, some are floats, and some are a str value of "free", which we need to conver to 0. When you handle monetary data, it is best to convert it to float (ex: 1.50)

In [21]:
def make_money(df: pd.DataFrame, column_name: str):

    # Use pandas replace to make "free" be 0
    df[column_name] = df[column_name].replace("free", 0)
    
    # Convert the column to float for currency
    df[column_name] = pd.to_numeric(df[column_name], errors='coerce').fillna(0).astype(float)
    
    return df

Call function to convert the price column to the proper format.

In [22]:
data = make_money(data, 'price')

Now lets glimpse at our clean data, check the datatypes, and look at some basic stats.

In [23]:
data.head()

,ID,name,author,narrator,releasedate,language,price,minutes,star_rating,ratings_count
0,294217,Shattered Hearts (German edition),Anne-MarieJungwirth,"FundaVanroy,SebastianFischer",11/19/21,german,401.0,527,NaN,0
1,253599,The Ultimate Colin Wilson,"ColinWilson,ColinStanley-editedby",LiamGerrard,05/14/19,USEnglish,703.0,918,NaN,0
2,210584,Bloody Confused!,ChuckCulpepper,AlexHyde-White,04/02/14,English,668.0,588,NaN,0
3,236792,Operation Greylord,"TerrenceHake,WayneKlatt",CharlesConstant,07/08/15,English,891.0,614,NaN,0
4,261210,The Little Book of Main Street Money,JonathanClements,SeanPratt,07/15/09,English,703.0,263,5.0,1


In [24]:
data.dtypes

ID                object
name              object
author            object
narrator          object
releasedate       object
language          object
price            float64
minutes            int64
star_rating      float64
ratings_count      int64
dtype: object

In [25]:
data.describe()

,price,minutes,star_rating,ratings_count
count,86998.000000,86998.000000,14988.000000,86998.000000
mean,560.848857,418.140992,4.642381,2.659406
std,335.903165,364.815476,0.732035,26.384400
min,0.000000,1.000000,1.000000,0.000000
25%,279.000000,143.000000,4.000000,0.000000
50%,585.000000,387.000000,5.000000,0.000000
75%,759.000000,585.000000,5.000000,0.000000
max,7198.000000,8595.000000,5.000000,985.000000


## 4. Export DataFrame to xlsx to use in Tableau

You did it! You cleaned a really gross dataset and now you're ready to use it in Tableau to create a visual analysis. We'll just simply use the pandas to_excel() function to write this locally to our machine. 

In [26]:
# Specify a file path for your machine
data.to_excel("/workspaces/audible_data_cleaning/audible_clean.xlsx", index=False)

## What you learned:

You just did a lot! Lets recap:

You loaded an xlsx file into python using the pandas read_excel()

Then, you utilized pandas, numpy, and regex and properly clean and manipulate your data. You used functions including:
* duplicated()
* drop_duplicates()
* isnull()
* dropna()
* astype()
* replace()
and many more!

Finally, you used exported your clean data to your local machine to then analyze in Tableau. You can now add data cleaning to your resume!